# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import pandas as pd
import numpy as np



# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [2]:
data = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')


Examine the `salaries` dataset using the `head` function below.

In [3]:
data.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
data.isnull() 

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,False,False,False,False,False,True,False,True
1,False,False,False,False,False,True,False,True
2,False,False,False,False,False,True,False,True
3,False,False,False,False,False,True,False,True
4,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...
33178,False,False,False,False,False,True,False,True
33179,False,False,False,False,False,True,False,True
33180,False,False,False,False,False,True,False,True
33181,False,False,False,False,False,True,False,True


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
df = data.set_index(['Salary or Hourly'])

print(len(df))

print(len(df.columns))

print(df.shape)

print(df.size)

33183
7
(33183, 7)
232281


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
df.groupby(by=['Department']).count()

,Name,Job Titles,Full or Part-Time,Typical Hours,Annual Salary,Hourly Rate
Department,,,,,,
ADMIN HEARNG,39,39,39,0,39,0
ANIMAL CONTRL,81,81,81,19,62,19
AVIATION,1629,1629,1629,1082,547,1082
BOARD OF ELECTION,107,107,107,0,107,0
BOARD OF ETHICS,8,8,8,0,8,0
BUDGET & MGMT,46,46,46,2,44,2
BUILDINGS,269,269,269,0,269,0
BUSINESS AFFAIRS,171,171,171,7,164,7
CITY CLERK,84,84,84,0,84,0


In [7]:
df['Department'].value_counts()

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [8]:
from scipy.stats import ttest_1samp

In [9]:
hourly = data[['Hourly Rate']].dropna()
mu =  30

In [10]:
hourly.head()

,Hourly Rate
4,19.86
6,46.10
7,35.60
10,2.65
18,17.68


In [11]:
hourly.mean()

Hourly Rate    32.788558
dtype: float64

In [12]:
ttest_1samp(hourly,30)

Ttest_1sampResult(statistic=array([20.61980579]), pvalue=array([4.32302405e-92]))

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [13]:
salaries = data.loc[data['Department'] == 'POLICE']

In [14]:
annual = salaries['Annual Salary'].dropna()

In [15]:
annual

0        101442.0
1         94122.0
9         93354.0
11        84054.0
12        87006.0
           ...   
33177     72510.0
33178     72510.0
33179     48078.0
33180     90024.0
33181     93354.0
Name: Annual Salary, Length: 13404, dtype: float64

In [16]:
ttest_1samp(annual, 86000)

Ttest_1sampResult(statistic=3.081997005712994, pvalue=0.0020603403550965137)

Using the `crosstab` function, find the department that has the most hourly workers. 

In [17]:
payment_per_department = pd.crosstab(data['Department'],data['Salary or Hourly'])


In [18]:
payment_per_department

Salary or Hourly,Hourly,Salary
Department,,
ADMIN HEARNG,0,39
ANIMAL CONTRL,19,62
AVIATION,1082,547
BOARD OF ELECTION,0,107
BOARD OF ETHICS,0,8
BUDGET & MGMT,2,44
BUILDINGS,0,269
BUSINESS AFFAIRS,7,164
CITY CLERK,0,84


In [19]:
payment_per_department.sort_values(by=['Hourly'], ascending = False)

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215
TRANSPORTN,725,415
PUBLIC LIBRARY,299,716
FAMILY & SUPPORT,287,328
CITY COUNCIL,64,347


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [21]:
street = data.loc[(data['Department'] == 'STREETS & SAN') & (data['Salary or Hourly'] == 'Hourly')]

In [22]:
street.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
21,"ABDUL-SHAKUR, TAHIR",GENERAL LABORER - DSS,STREETS & SAN,F,Hourly,40.0,NaN,21.43
24,"ABERCROMBIE, TIMOTHY",MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
36,"ABRAMS, DANIELLE T",SANITATION LABORER,STREETS & SAN,F,Hourly,40.0,NaN,36.21
39,"ABRAMS, SAMUEL A",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60


In [23]:
street = street['Hourly Rate']

In [24]:
street.head()

7     35.60
21    21.43
24    35.60
36    36.21
39    35.60
Name: Hourly Rate, dtype: float64

In [25]:
ttest_1samp(street, 35)

Ttest_1sampResult(statistic=-9.567447887848152, pvalue=3.3378530564707717e-21)

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [52]:
mean_street = data['Hourly Rate'].dropna().mean()

deg_freedom = (data['Hourly Rate'].count() - 1)

In [53]:
from scipy import stats

In [54]:
std_err = stats.sem(data['Hourly Rate'].dropna())

In [55]:
t = stats.t.interval(0.95, loc = mean_street, scale = std_err, df = deg_freedom)
print(t)

(32.52345834488529, 33.05365708767727)


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [51]:
annual_police = data.loc[(data['Department'] == 'POLICE') & (data['Salary or Hourly'] == 'Annual Salary')]

In [51]:
mean_police = data['Annual Salary'].dropna().mean()

In [51]:
deg_freedom = (data['Annual Salary'].count() - 1)

In [51]:
std_err = stats.sem(data['Annual Salary'].dropna())

In [51]:
t = stats.t.interval(0.95, loc = mean_police, scale = std_err, df = deg_freedom)
print(t)

(86526.99656774197, 87047.0030125611)


# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

